**Train an NMF model on several types of 1D pixel histograms and study the combined prediction**

Note: very similar to autoencoder_combine.ipynb (in fact this script started as a copy of that one), but here the used model is not an autoencoder but an NMF model.

In [ ]:
### imports

# external modules
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import HistStruct
importlib.reload(HistStruct)
import NMFClassifier
importlib.reload(NMFClassifier)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)

In [ ]:
### define run properties
# in this cell all major run properties are going to be set,
# e.g. what runs to train on and what runs to test on

# define a list of good 'reference' runs (found by eye)
# should be replaced at some point by the reference runs defined by the DQM/DC team.
goodrunsls = {'2017':
                {
                "297056":[[-1]],
                "297177":[[-1]],
                "301449":[[-1]],
                }
              }

# define core test set of clearly bad runs (found by eye)
badrunsls = {'2017':
                {
                "297287":[[-1]],
                "297288":[[-1]],
                "297289":[[-1]],
                "299316":[[-1]],
                "299324":[[-1]],
                "299326":[[-1]],
                "301086":[[88,126]] # only bad for size_PXDisk_+1 -> maybe do not use for now (unclear what are real anomalies)
                }
            }

# set year to use
year = '2017'

# set histogram names to use 
histnames = [
            'chargeInner_PXLayer_2',
             'chargeInner_PXLayer_3',
             'charge_PXDisk_+1','charge_PXDisk_+2','charge_PXDisk_+3',
             'size_PXLayer_1','size_PXLayer_2',
             'size_PXLayer_3'
            ]

# set whether to train globally or locally
training_mode = 'global'

if training_mode == 'global':
    runsls_training = None # use none to not add a mask for training (can e.g. use DCS-bit on mask)
    runsls_good = None # use none to not add a mask for good runs (can e.g. use templates)
    runsls_bad = badrunsls[year] # predefined bad runs
    print('selected runs/lumisections for training: all')
    
elif training_mode == 'local':
    # train locally on a small set of runs
    # for now on n runs preceding a chosen application run,
    # to be extended with choosing reference runs.
    
    # select application run
    available_runs = dfu.get_runs( dfu.select_dcson( csvu.read_csv('../data/DF'+year+'_'+histnames[0]+'.csv') ) )
    run_application = 305351
    run_application_index = available_runs.index(run_application)
    # select training set
    ntraining = 5
    runsls_training = jsonu.tuplelist_to_jsondict([(el,[-1]) for el in available_runs[run_application_index-ntraining:run_application_index]])
    runsls_bad = badrunsls[year]
    runsls_good = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)

In [ ]:
### read the data based on the configuration defined above

readnew = True
save = False

if readnew:
    
    # add the histograms
    histstruct = HistStruct.HistStruct()
    # loop over the histogram types to take into account
    for histname in histnames:
        print('adding {}...'.format(histname))
        # read the histograms from the csv file
        filename = '../data/DF2017_'+histname+'.csv'
        df = csvu.read_csv( filename )
        # in case of local training, we can remove most of the histograms
        if( runsls_training is not None and runsls_good is not None and runsls_bad is not None ):
            runsls_total = {k: v for d in (runsls_training, runsls_good, runsls_bad) for k, v in d.items()}
            df = dfu.select_runsls( df, runsls_total )
        histstruct.add_dataframe( df )
    print('found {} histograms'.format(len(histstruct.runnbs)))
    
    # add masks
    histstruct.add_dcsonjson_mask( 'dcson' )
    histstruct.add_goldenjson_mask('golden' )
    histstruct.add_hightstat_mask( 'highstat' )
    if runsls_training is not None: histstruct.add_json_mask( 'training', runsls_training )
    if runsls_good is not None: histstruct.add_json_mask( 'good', runsls_good )
    nbadruns = 0
    if runsls_bad is not None:
        histstruct.add_json_mask( 'bad', runsls_bad )
        # special case for bad runs: add a mask per run (different bad runs have different characteristics)
        nbadruns = len(runsls_bad.keys())
        for i,badrun in enumerate(runsls_bad.keys()):
            histstruct.add_json_mask( 'bad{}'.format(i), {badrun:runsls_bad[badrun]} )
    
    if save:
        histstruct.save( 'test.pkl' )
        
if not readnew:
    
    histstruct = HistStruct.HistStruct.load( 'test.pkl' )
    nbadruns = len([name for name in list(histstruct.masks.keys()) if 'bad' in name])
    
print('created a histstruct with the following properties:')
print('- number of histogram types: {}'.format(len(histstruct.histnames)))
print('- number of lumisections: {}'.format(len(histstruct.lsnbs)))
print('- masks: {}'.format(list(histstruct.masks.keys())))

In [ ]:
### plot the training and/or test sets
# especially useful if running in local mode to check if training set is relevant for targeted application run,
# and if the application run contains anomalies

skipthiscell = True

if( training_mode=='local' and not skipthiscell ):
    
    # training and application runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','training'],['highstat','good']],
                                labellist = ['training','testing'],
                                colorlist = ['blue','green']
                              )
    
    # application run and bad test runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['bad']],
                                labellist = ['good','bad'],
                                colorlist = ['green','red']
                              )

In [ ]:
### extend the training set using artificial data

extendtraining = False

if extendtraining:
    histstruct.exthistograms['training'] = {}
    for histname,hists in histstruct.get_histograms( masknames=['dcson','highstat','training'] ).items():
        print('generating artificial training data for '+histname)
        histstruct.exthistograms['training'][histname] = gdu.upsample_hist_set(hists,5e4)
        print(' -> generated {} histograms'.format(len(histstruct.exthistograms['training'][histname])))

In [ ]:
### define and train an NMF model for each element

trainnew = True
save = False # ignored if trainnew is False
name_ext = '2017_global_training_dcson_highstat_ncomponents3'

if trainnew:
    for histname in histstruct.histnames:
        print('building NMF model for histogram type {}'.format(histname))
        if training_mode=='local': hists_train = histstruct.get_histograms( histname=histname, masknames=['dcson','highstat','training'] )
        elif training_mode=='global':
            # use all available data for training (with DCS-on and statistics selection)
            #hists_train = histstruct.get_histograms( histname=histname, masknames=['dcson','highstat'] )
            # this can however take a long time... alternatively, use averaged histograms for training
            # warning: this could lead to bias since averaged histograms will be 'artificially good'
            #hists_train = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat'] ), int(1e3) )
            # alternatively, train on randomly chosen histograms from training set
            hists_train = histstruct.get_histograms( histname=histname, masknames=['dcson','highstat'] )
            random_indices = np.random.choice(len(hists_train), size=int(5e4), replace=False)
            hists_train = hists_train[random_indices]
        if extendtraining: hists_train = histstruct.exthistograms['training'][histname]
        print('size of training set: {}'.format(hists_train.shape))
        classifier = NMFClassifier.NMFClassifier( hists_train, ncomponents=3, nmax=10 )
        histstruct.add_classifier( histname, classifier, evaluate=False )
        if save: 
            path = '../models/nmfclassifier_{}_{}.pkl'.format(histname,name_ext)
            with open(path,'wb') as f:
                pickle.dump(classifier,f)
                
else:
    for histname in histstruct.histnames:
        path = '../models/nmfclassifier_{}_{}.pkl'.format(histname,name_ext)
        with open(path,'rb') as f:
            classifier = pickle.load(f)
        histstruct.add_classifier( histname, classifier, evaluate=False )

In [ ]:
### plot NMF model components

# initializations
ncols = min(4,len(histnames))
nrows = int(math.ceil(len(histnames)/ncols))
fig,axs = plt.subplots(nrows,ncols,figsize=(6*ncols,6*nrows),squeeze=False)
# loop over all histogram types
for j,histname in enumerate(histstruct.histnames):
    # get the histograms to plot
    hists = histstruct.classifiers[histname].get_components()
    pu.plot_hists_multi(hists, fig=fig, ax=axs[int(j/ncols),j%ncols],
                        title=histname, colorlist=list(range(len(hists))) )

In [ ]:
### reset NMF model loss parameters without re-training

for histname in histstruct.histnames:
    histstruct.classifiers[histname].set_nmax( 10 )
    #histstruct.classifiers[histname].set_loss_type( 'chi2' )

In [ ]:
### evaluate the models on all histograms in the (non-extended) histstruct

mse_train = []
mse_good = []
mse_bad = []
for i in range(nbadruns): mse_bad.append([])
for histname in histstruct.histnames:
    print('evaluating model for '+histname)
    histstruct.evaluate_classifier(histname)
    # get mse for training set
    if 'training' in histstruct.masks.keys():
        thismse = histstruct.get_scores( histname=histname, masknames=['dcson','highstat', 'training'] )
    else:
        thismse = histstruct.get_scores( histname=histname, masknames=['dcson','highstat'] )
    mse_train.append( thismse )
    # get mse for good set
    if 'good' in histstruct.masks.keys(): 
        thismse = histstruct.get_scores( histname=histname, masknames=['dcson','highstat','good'] )
    else:
        hists_good = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat']), 1000 )
        thismse = histstruct.classifiers[histname].evaluate( hists_good )
    mse_good.append( thismse )
    # get mse for bad sets
    for i in range(nbadruns):
        thismse = histstruct.get_scores( histname=histname, masknames=['dcson','highstat','bad{}'.format(i)] )
        mse_bad[i].append( thismse )
        
# transform to arrays with correct shape
mse_train = np.array(mse_train)
mse_train = np.transpose(mse_train)
print('found mse array for training set of following shape: {}'.format(mse_train.shape))
mse_good = np.array(mse_good)
mse_good = np.transpose(mse_good)
print('found mse array for good set of following shape: {}'.format(mse_good.shape))
for i in range(nbadruns):
    mse_bad[i] = np.array(mse_bad[i])
    mse_bad[i] = np.transpose(mse_bad[i])
    print('found mse array for bad set of following shape: {}'.format(mse_bad[i].shape))

In [ ]:
### plot the multidemensional mse and fit a distribution

importlib.reload(HyperRectangleFitter)
importlib.reload(pu)

dimslist = []
fitfunclist = []
nhisttypes = len(histstruct.histnames)
for i in range(0,nhisttypes-1):
    for j in range(i+1,nhisttypes):
        dimslist.append((i,j))

plt.close('all')
(npoints,ndims) = mse_train.shape
scott_bw = npoints**(-1./(ndims+4))
bw_method = 20*scott_bw
for dims in dimslist:
   
    thismse = mse_train[:,dims]
    if training_mode=='global': 
        fitfunc = SeminormalFitter.SeminormalFitter(thismse)
        #fitfunc = HyperRectangleFitter.HyperRectangleFitter( thismse, 0.001, 'up', verbose=True )
    else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(thismse,bw_method=bw_method)
    fig,ax = pu.plot_fit_2d(thismse, fitfunc=fitfunc, logprob=True, clipprob=True,
                    onlycontour=False, xlims=30, ylims=30, 
                    onlypositive=True, xaxtitle=histstruct.histnames[dims[0]], 
                    yaxtitle=histstruct.histnames[dims[1]], transparency=0.5)
    plt.show()
    #plt.close('all') # release plot memory
    fitfunclist.append(fitfunc)
    
if training_mode=='global': 
    fitfunc = SeminormalFitter.SeminormalFitter(mse_train)
    #fitfunc = HyperRectangleFitter.HyperRectangleFitter( mse_train, 0.001, 'up', verbose=True )
else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(mse_train,bw_method=bw_method)

In [ ]:
### extend the test set using artificial data generation and evaluate the model on the extended test set

skipthiscell = False # to prevent running this cell by accident

if not skipthiscell:
    
    histstruct.exthistograms['good'] = {}
    for i in range(nbadruns): histstruct.exthistograms['bad{}'.format(i)] = {}
    for histname in histstruct.histnames:
        print('generating data for '+histname)
        if 'good' in histstruct.masks.keys():
            goodhists = histstruct.get_histograms( histname=histname,masknames=['dcson','highstat','good'] )
        else:
            goodhists = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat'] ), 15 )
        histstruct.exthistograms['good'][histname] = gdu.upsample_hist_set( goodhists,
                                                    figname='',ntarget=nbadruns*5e3,fourierstdfactor=20.)
        # alternative: copy original good set (e.g. for using resampled bad but original good)
        #histstruct.exthistograms['good'][name] = goodhists
        for i in range(nbadruns):
            badhists = histstruct.get_histograms( histname=histname,masknames=['dcson','highstat','bad{}'.format(i)] )
            histstruct.exthistograms['bad{}'.format(i)][histname] = gdu.upsample_hist_set(
                badhists,figname='',ntarget=5e3,fourierstdfactor=20.)
            
    mse_good_ext = []
    mse_bad_ext = []
    for i in range(nbadruns): mse_bad_ext.append( [] )

    for histname in histstruct.histnames:
        print('evaluating: '+histname)
        # calculate mse for good test set
        mse = histstruct.classifiers[histname].evaluate( histstruct.exthistograms['good'][histname] )
        mse_good_ext.append(mse)
        for i in range(nbadruns):
            # calculate mse
            mse = histstruct.classifiers[histname].evaluate( histstruct.exthistograms['bad{}'.format(i)][histname] )
            mse_bad_ext[i].append(mse)
    
    # transform to arrays with correct shape
    mse_good_ext = np.array(mse_good_ext)
    mse_good_ext = np.transpose(mse_good_ext)
    print('found mse array for good set of following shape: {}'.format(mse_good_ext.shape))
    for i in range(nbadruns):
        mse_bad_ext[i] = np.array(mse_bad_ext[i])
        mse_bad_ext[i] = np.transpose(mse_bad_ext[i])
        print('found mse array for bad set of following shape: {}'.format(mse_bad_ext[i].shape))

In [ ]:
### (re-)define the test set

use_ext = True

mse_good_eval = mse_good
mse_bad_eval = mse_bad
if use_ext:
    mse_good_eval = mse_good_ext
    mse_bad_eval = mse_bad_ext
    
# subselect only specific bad sets for quick checking
# note: not very clean, e.g. watch out with using nbadsets after this cell...
#mse_bad_eval = [mse_bad_eval[6]]

In [ ]:
### make a new plot of probability contours and overlay data points
### (only 2D projections!)
  
doplot = False

if doplot:
    plt.close('all')
    colorlist = ['red','lightcoral','firebrick','chocolate','fuchsia','orange','purple']
    if len(colorlist)<len(mse_bad_eval):
        raise Exception('ERROR: need more colors...')

    for dims,partialfitfunc in zip(dimslist,fitfunclist):
        fig,ax = pu.plot_fit_2d(mse_train[:,dims], fitfunc=partialfitfunc, logprob=True, clipprob=True, 
                    onlycontour=True, xlims=30, ylims=30, 
                    onlypositive=True, xaxtitle=histstruct.histnames[dims[0]], 
                    yaxtitle=histstruct.histnames[dims[1]],
                    transparency=0.5)
        for j in range(len(mse_bad_eval)):
            ax.plot(mse_bad_eval[j][:,dims[0]],mse_bad_eval[j][:,dims[1]],
                '.',color=colorlist[j],markersize=4)
        ax.plot(mse_good_eval[:,dims[0]],mse_good_eval[:,dims[1]],'.',color='blue',markersize=4)
        plt.show()
    

# get the minimum log probability of histograms in good set
print('--- good lumesections ---')
logprob_good = np.log(fitfunc.pdf(mse_good_eval))
print('length of log prob array: '+str(len(logprob_good)))
print('minimum of log prob: '+str(np.min(logprob_good)))
#print(sorted(logprob_good))
print('--- bad lumisections ---')
logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]
#for lp in logprob_bad_parts: print(str(sorted(lp))+'\n\n')
logprob_bad = np.concatenate(tuple(logprob_bad_parts))
print('length of log prob array: '+str(len(logprob_bad)))
print('maximum of log prob: '+str(np.max(logprob_bad)))
#print(sorted(logprob_good))
#print(sorted(logprob_bad))
#print(logprob_bad)

In [ ]:
### make a roc curve based on the test results above
# note: smaller logprob = less probable = more outlier = more anomalous
# so if anomalies are signal and good histograms are background, -logprob is a suitable score definition,
# since everything above a certain threshold will be considered signal and below it background.

labels_good = np.zeros(len(logprob_good)) # background: label = 0
labels_bad = np.ones(len(logprob_bad)) # signal: label = 1

labels = np.concatenate(tuple([labels_good,labels_bad]))
scores = np.concatenate(tuple([-logprob_good,-logprob_bad]))
scores = aeu.clip_scores(scores)

pu.plot_score_dist(scores, labels, siglabel='anomalous', sigcolor='r', 
                   bcklabel='good', bckcolor='g', 
                   nbins=200, normalize=True,
                   xaxtitle='negative logarithmic probability',
                   yaxtitle='number of lumisections (normalized)')

auc = aeu.get_roc(scores, labels, mode='geom', doprint=True)

logprob_threshold = 65 # everything below this logprob will be considered signal (i.e. anomalous)
aeu.get_confusion_matrix(scores,labels,-logprob_threshold)

In [ ]:
### investigate particular lumisections

mode = 'ls'
run = 306139
ls = 1112 # ignored if mode is 'run'
masknames = ['dcson','highstat'] # only used for mse distribution is mode is 'ls'
plot_mse = True # ignored if mode is 'run'

#print(histstruct.runnbs[:10])
#print(histstruct.lsnbs[:10])

# define reference histograms
refhists = {}
for histname in histstruct.histnames:
    if( 'good' in histstruct.masks.keys() ): 
        #refhists[histname] = histstruct.get_histograms(masknames=['highstat','dcson','good'])
        refhists[histname] = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['highstat','dcson','good']), 15 )
    else: 
        refhists[histname] = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat'] ), 15 )

if mode=='ls':
    # plot this particular run/ls
    _ = histstruct.plot_ls( run, ls, recohist='auto', refhists=refhists )
    # get mse and logprob for this run/ls
    msepoint = histstruct.get_scores_ls( run, ls )
    msepointarray = np.array([msepoint[histname] for histname in histstruct.histnames])
    logprob = np.log(fitfunc.pdf(np.array([msepointarray])))
    print('-------------')
    print('MSE values:')
    for histname in histstruct.histnames: print('{} : {}'.format(histname,msepoint[histname]))
    print('-------------')
    print('logprob: '+str(logprob))
    msepoint = histstruct.get_scores_ls( run, ls )
    # plot mse distribution
    if plot_mse:
        for histname in histnames:
            mses = histstruct.get_scores( histname=histname, masknames=masknames )
            nmses = len(mses)
            labels = np.zeros(nmses)
            mses = np.concatenate((mses,np.ones(1)*msepoint[histname]))
            labels = np.concatenate((labels,np.ones(1)))
            pu.plot_score_dist( mses, labels, nbins=1000, normalize=True,
                        siglabel='this lumisection', bcklabel='all lumisections',
                        title=histname )

if mode=='run':
    # plot given run
    runnbs = histstruct.get_runnbs( masknames=masknames )
    lsnbs = histstruct.get_lsnbs( masknames=masknames )
    runsel = np.where(runnbs==run)
    lsnbs = lsnbs[runsel]
    print('plotting {} lumisections...'.format(len(lsnbs)))
    for lsnb in lsnbs:
        fig,ax = histstruct.plot_ls(run, lsnb, recohist='auto', refhists=refhists )
        plt.show()
        msepoint = histstruct.get_scores_ls( run, lsnb )
        msepointarray = np.array([msepoint[histname] for histname in histstruct.histnames])
        logprob = np.log(fitfunc.pdf(np.array([msepointarray])))
        print('-------------')
        print('MSE values:')
        for histname in histstruct.histnames: print('{} : {}'.format(histname,msepoint[histname]))
        print('-------------')
        print('logprob: '+str(logprob))

In [ ]:
### investigate how the method performs on the golden/custom test set

#evaljson = jsonu.loadjson('utils/json_pixel_good_201201.json')
#histstruct.add_json_mask( 'pixelgood', evaljson )
masks_eval = ['golden', 'highstat']
lsnbs_eval = histstruct.get_lsnbs( masknames=masks_eval )
runnbs_eval = histstruct.get_runnbs( masknames=masks_eval )
mse_eval_dict = histstruct.get_scores( masknames=masks_eval )
mse_eval = []
for histname in histstruct.histnames:
    mse_eval.append( mse_eval_dict[histname] )
mse_eval = np.array(mse_eval)
mse_eval = np.transpose(mse_eval)
print('found mse array for evaluation set of following shape: {}'.format(mse_eval.shape))
logprob_eval = np.log(fitfunc.pdf(mse_eval))

def get_runsls_inrange(logprob,runnbs,lsnbs,logprob_up=None,logprob_down=None):
    # get a list of tuples of (run number, ls number) corresponding to ls with log probability within a given range
    # - logprob, runnbs and lsnbs are equally long 1D arrays
    # - logprob_up and logprob_down are upper and lower thresholds
    #     if both are not None, the lumisections with logprob between the boundaries are returned
    #     if logprob_up is None, the lumisections with logprob > logprob_down are returned
    #     if logprob_down is None, the lumisections with logprob < logprob_up are returned
    indices = np.array([])
    if logprob_down is None:
        indices = np.nonzero(logprob<logprob_up)[0]
    elif logprob_up is None:
        indices = np.nonzero(logprob>logprob_down)[0]
    else:
        indices = np.nonzero((logprob>logprob_down) & (logprob<logprob_up))[0]
    runsinrange = runnbs[indices]
    lsinrange = lsnbs[indices]
    runslsinrange = []
    for rr,lsr in zip(runsinrange,lsinrange):
        runslsinrange.append((int(rr),int(lsr)))
    return {'indices':indices,'runslsinrange':runslsinrange}

logup = 65
logdown = None
temp = get_runsls_inrange(logprob_eval, runnbs_eval, lsnbs_eval,
                          logprob_up = logup, logprob_down = logdown)

runslsinrange = temp['runslsinrange']
print('{} out of {} LS are within these boundaries'.format(len(runslsinrange),len(logprob_eval)))

# make plots
nplotsmax = 100
from matplotlib.backends.backend_pdf import PdfPages
pdf = PdfPages('nmf_combine_output.pdf')
for i,(runnb,lsnb) in enumerate(runslsinrange):
    if i>=nplotsmax:
        print('maximum number of plots reached')
        break
    print('------------------------')
    fig,axs = histstruct.plot_ls( runnb, lsnb, recohist='auto', refhists=refhists)
    fig.show()
    pdf.savefig(fig)
    # only for 2 dimensions: extra contour plot
    #if nhisttypes != 2: continue
    #fig,ax = plt.subplots()
    #contourplot = ax.contourf(x, y, np.log(fitfunc.pdfgrid(pos)))
    #plt.colorbar(contourplot)
    #ax.plot(msepoint[0],msepoint[1],'.k',markersize=10)
    #ax.set_xlim((0.,xlim))
    #ax.set_ylim((0.,ylim))
pdf.close()